# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import pandas as pd
import numpy as np
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

from sqlalchemy import create_engine

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster_messages.db')
df = pd.read_sql_table('Disaster_messages', engine)
X = df['message']
Y = df.drop(['id','message','original', 'genre','is_dupe'], axis=1)

In [3]:
np.sum(df.iloc[:,4:],axis=1).tolist()

[1,
 5,
 1,
 8,
 1,
 0,
 3,
 6,
 0,
 6,
 15,
 0,
 5,
 5,
 1,
 9,
 7,
 0,
 0,
 3,
 1,
 5,
 5,
 4,
 1,
 0,
 8,
 9,
 6,
 5,
 0,
 6,
 0,
 1,
 4,
 1,
 5,
 5,
 5,
 7,
 10,
 4,
 4,
 8,
 5,
 2,
 6,
 5,
 5,
 4,
 1,
 6,
 0,
 1,
 5,
 1,
 10,
 5,
 6,
 5,
 5,
 5,
 4,
 7,
 9,
 4,
 0,
 1,
 2,
 6,
 5,
 5,
 8,
 1,
 5,
 2,
 3,
 7,
 6,
 0,
 0,
 6,
 0,
 0,
 16,
 1,
 7,
 0,
 8,
 3,
 5,
 7,
 6,
 6,
 4,
 3,
 0,
 0,
 1,
 8,
 6,
 5,
 1,
 0,
 0,
 8,
 6,
 11,
 9,
 3,
 1,
 6,
 4,
 6,
 13,
 0,
 6,
 2,
 6,
 5,
 4,
 0,
 5,
 11,
 1,
 9,
 15,
 6,
 5,
 6,
 5,
 0,
 6,
 5,
 1,
 0,
 1,
 5,
 10,
 5,
 1,
 1,
 0,
 0,
 1,
 1,
 5,
 0,
 5,
 5,
 5,
 11,
 7,
 15,
 6,
 5,
 0,
 1,
 5,
 5,
 1,
 3,
 5,
 4,
 6,
 8,
 4,
 6,
 4,
 7,
 14,
 7,
 8,
 5,
 8,
 0,
 1,
 4,
 0,
 5,
 1,
 5,
 5,
 6,
 5,
 5,
 8,
 6,
 5,
 6,
 7,
 4,
 7,
 1,
 6,
 9,
 5,
 6,
 6,
 7,
 8,
 1,
 0,
 5,
 6,
 11,
 3,
 4,
 8,
 6,
 8,
 1,
 4,
 7,
 5,
 3,
 0,
 6,
 1,
 2,
 10,
 1,
 6,
 1,
 5,
 10,
 8,
 9,
 4,
 9,
 5,
 7,
 1,
 7,
 0,
 1,
 5,
 6,
 0,
 1,
 11,
 6,
 5,
 0,
 2,
 1,


In [4]:
categories=list(Y)

In [5]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26247 entries, 0 to 26246
Data columns (total 36 columns):
related                   26247 non-null int64
request                   26247 non-null int64
offer                     26247 non-null int64
aid_related               26247 non-null int64
medical_help              26247 non-null int64
medical_products          26247 non-null int64
search_and_rescue         26247 non-null int64
security                  26247 non-null int64
military                  26247 non-null int64
child_alone               26247 non-null int64
water                     26247 non-null int64
food                      26247 non-null int64
shelter                   26247 non-null int64
clothing                  26247 non-null int64
money                     26247 non-null int64
missing_people            26247 non-null int64
refugees                  26247 non-null int64
death                     26247 non-null int64
other_aid                 26247 non-null int6

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokens=word_tokenize(text)
    Lemmatizer=WordNetLemmatizer()
    tokens_lem=[Lemmatizer.lemmatize(t).lower().strip() for t in tokens]
    return tokens_lem

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
def model_pipeline():
    pipeline =Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))   
        ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=101)

In [9]:
model=model_pipeline()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

In [10]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
accuracy = (y_pred == Y_test).mean()
accuracy

related                   0.784571
request                   0.881524
offer                     0.995048
aid_related               0.741714
medical_help              0.921143
medical_products          0.953714
search_and_rescue         0.977143
security                  0.982476
military                  0.969905
child_alone               1.000000
water                     0.948571
food                      0.913714
shelter                   0.922667
clothing                  0.984000
money                     0.976571
missing_people            0.987619
refugees                  0.969143
death                     0.959619
other_aid                 0.875429
infrastructure_related    0.931810
transport                 0.955810
buildings                 0.954476
electricity               0.980381
tools                     0.995048
hospitals                 0.989143
shops                     0.995429
aid_centers               0.987619
other_infrastructure      0.954286
weather_related     

In [12]:
for i in range(36):
    print("Category:", categories[i],"\n", classification_report(Y_test.values[:, i], y_pred[:, i]))

Category: related 
              precision    recall  f1-score   support

          0       0.61      0.35      0.44      1254
          1       0.81      0.93      0.87      3960
          2       0.57      0.11      0.19        36

avg / total       0.76      0.78      0.76      5250

Category: request 
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      4386
          1       0.80      0.38      0.51       864

avg / total       0.87      0.88      0.86      5250

Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5224
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5250

Category: aid_related 
              precision    recall  f1-score   support

          0       0.74      0.88      0.80      3147
          1       0.75      0.54      0.62      2103

avg / total       0.74      0.74      0.73      5250

Categ

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
def build_model():
    '''
    Finds the best parameters for RF model
    param: model
    param: X_train
    param: Y_train
    return: best set of parameters
    '''
    pipeline =Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))   
        ])
    parameters = {
        'clf__estimator__n_estimators': [50, 100, 150],
        'clf__estimator__max_features': ['sqrt',],
        'clf__estimator__criterion': ['entropy', 'gini']
    }

    cv = GridSearchCV(pipeline, param_grid = parameters, verbose=1)

    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
model=build_model()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 137.7min finished


In [16]:
model.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_features': 'sqrt',
 'clf__estimator__n_estimators': 100}

In [17]:
accuracy = (y_pred == Y_test).mean()
accuracy

related                   0.789143
request                   0.894476
offer                     0.995048
aid_related               0.780381
medical_help              0.920571
medical_products          0.954095
search_and_rescue         0.976381
security                  0.982476
military                  0.970476
child_alone               1.000000
water                     0.946667
food                      0.929333
shelter                   0.930476
clothing                  0.984381
money                     0.977524
missing_people            0.987810
refugees                  0.968381
death                     0.962286
other_aid                 0.875810
infrastructure_related    0.932190
transport                 0.955048
buildings                 0.953714
electricity               0.980190
tools                     0.995048
hospitals                 0.989143
shops                     0.995429
aid_centers               0.987619
other_infrastructure      0.954095
weather_related     

In [18]:
for i in range(36):
    print("Category:", categories[i],"\n", classification_report(Y_test.values[:, i], y_pred[:, i]))
#we see improvements in a few categories like request, aid_related, food weather related and direct report

Category: related 
              precision    recall  f1-score   support

          0       0.70      0.25      0.37      1254
          1       0.80      0.97      0.87      3960
          2       0.40      0.06      0.10        36

avg / total       0.77      0.79      0.75      5250

Category: request 
              precision    recall  f1-score   support

          0       0.90      0.98      0.94      4386
          1       0.85      0.44      0.58       864

avg / total       0.89      0.89      0.88      5250

Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5224
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5250

Category: aid_related 
              precision    recall  f1-score   support

          0       0.78      0.89      0.83      3147
          1       0.79      0.62      0.69      2103

avg / total       0.78      0.78      0.77      5250

Categ

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [8]:
def new_model():
    '''
    Finds the best parameters for RF model
    param: model
    param: X_train
    param: Y_train
    return: best set of parameters
    '''
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(criterion='gini',max_features='sqrt',n_estimators=100)))   
        ])
    parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
         'vect__max_df': (0.75, 1.0)
    }

    cv = GridSearchCV(pipeline, param_grid = parameters, verbose=1,n_jobs=-1)

    return cv

In [9]:
newmodel=new_model()
newmodel.fit(X_train, Y_train)
y_pred = newmodel.predict(X_test)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 44.5min finished


### 9. Export your model as a pickle file

In [10]:
with open('newmodel.pkl', 'wb') as file:
    pickle.dump(newmodel, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:

# import libraries
import sys
import pandas as pd
import numpy as np
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

from sqlalchemy import create_engine

import pickle

def load_data(filepath):
    ''''
    Load data function
    Inputs:
    filepath: path to SQLite db
    Outputs:
    X:features 
    Y:label
    category_names: types of disaster
    ''''
    engine = create_engine('sqlite:///'+filepath)
    df = pd.read_sql_table('Disaster_messages', engine)
    X = df['message']
    Y = df.drop(['id','message','original', 'genre','is_dupe'], axis=1)
    category_names=Y.columns
    return X,Y,category_names

def tokenize(text):
    ''''
    inputs: 
    text: list of text messages
    outputs:
    tokens_lem: tokenized text
    ''''
    tokens=word_tokenize(text)
    Lemmatizer=WordNetLemmatizer()
    tokens_lem=[Lemmatizer.lemmatize(t).lower().strip() for t in tokens]
    return tokens_lem

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    
def build_model():
    '''
    Finds the best parameters for RF model
    param: model
    param: X_train
    param: Y_train
    return: best set of parameters
    '''
    pipeline =Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))   
        ])
    parameters = {
        'clf__estimator__n_estimators': [50, 100, 150],
        'clf__estimator__max_features': ['sqrt',],
        'clf__estimator__criterion': ['entropy', 'gini']
    }

    cv = GridSearchCV(pipeline, param_grid = parameters, verbose=1)

    return cv

def evaulate_model(model,X_test,Y_test,category_names):
    ''''
    This function will evaluate the model performance
    Input:
    X_test:test features
    y_test:test target
    category_names: labels names
    ''''
    y_pred = model.predict(X_test)
    for i in range(36):
        print("Category:", categories[i],"\n", classification_report(Y_test.values[:, i], y_pred[:, i]))
        
def save_model(model,model_path):
    ''''
    This function saves the model in a pkl file
    Input:
    model
    model_path
    Output: 
    pickle file of the saved model
    ''''
    with open('newmodel.pkl', 'wb') as file:
        pickle.dump(newmodel, file)
        
def main():
        X, Y, category_names = load_data('Disaster_messages.db')
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=101)
        model = build_model()
        model.fit(X_train, Y_train)
        evaluate_model(model, X_test, Y_test, category_names)
        save_model(model, model_filepath)